In [ ]:
using Polyhedra, CDDLib
using DrakeVisualizer
import ColorTypes: RGBA
using PyCall
path_to_my_module = "/home/manuelli/drake-mpc/src/boxrobots/src"
push!(LOAD_PATH, path_to_my_module)
reload("BoxRobots")
import BoxRobots
using DrakeVisualizer
import ColorTypes: RGBA
using Polyhedra: SimpleHRepresentation

br = BoxRobots

In [ ]:
# unshift!(PyVector(pyimport("sys")["path"]), "../")
@pyimport boxatlas
@pyimport boxatlas.boxatlascontroller as boxatlascontroller
@pyimport boxatlas.contactstabilizationutils as csu
CSU = csu.ContactStabilizationUtils

In [ ]:
# attempt to visualize a state
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window();
vis = Visualizer()
delete!(vis)
env, robot = br.make_robot_and_environment()
robot_state = br.make_robot_state()
robot_state.centroidal_dynamics_state.vel[1] = -1.75
br.draw_environment(vis, env)
mass = robot.mass
gravity = robot.gravity
vis_options = br.BoxRobotVisualizerOptions(force_arrow_normalizer=mass*abs(gravity[end]))

# QPInnerLoopController

In [ ]:
# setup MIQP controller so we can get python controller out
miqp_controller = br.MIQPController()
robot_state = br.make_robot_state()
robot_state.centroidal_dynamics_state.vel[1] = -1.75
python_controller = miqp_controller.python_controller
robot_python = python_controller[:robot]
state_python = br.convert_box_robot_state_to_python(robot_python, robot_state)
opt = python_controller[:construct_contact_stabilization_optimization](state_python)
soln_data = pycall(opt[:solve], PyObject)

# Simulate QPInnerLoopController

In [ ]:
t_start = 0.0
dt = 0.05
duration = 1.5
qp_controller = br.QPInnerLoopController(python_controller, t_start,
    robot_state, soln_data)
@time traj = br.simulate_tspan(robot, qp_controller, robot_state, dt, duration)
br.playback_trajectory(vis, traj; options=vis_options)

In [ ]:
br.playback_trajectory(vis, traj, options=vis_options, playback_speed=1.0)

# Simulate MIQP Controller

In [ ]:

miqp_controller = br.MIQPController()
robot_state = br.make_robot_state()
robot_state.centroidal_dynamics_state.vel[1] = -1.75
python_controller = miqp_controller.python_controller
robot_python = python_controller[:robot]
state_python = br.convert_box_robot_state_to_python(robot_python, robot_state)
opt = python_controller[:construct_contact_stabilization_optimization](state_python)
soln_data = pycall(opt[:solve], PyObject)

# construct the SimpleInnerLoopController
t_start = 0.
inner_loop_controller = br.SimpleInnerLoopController(t_start, soln_data)

t = 0.0
dt = 0.025
duration = 1.0
@time traj = br.simulate_tspan(robot, inner_loop_controller, robot_state, dt, duration)
br.playback_trajectory(vis, traj; options=vis_options)


In [ ]:
# extract solve times from the trajectory of SimulationData
function get_solve_time(data)
    return data.controller_data.solve_time
end

function get_contact_array(data)
    return data.state.limb_states[:left_hand].in_contact 
end

solve_time_array = get_solve_time.(traj.data)
contact_array = get_contact_array.(traj.data)

In [ ]:
using Plots
# Use the GR backend for Plots.jl, because it's fast
gr()

In [ ]:
plot(traj.time, solve_time_array, xlabel="time", label="solve time")
# plot!(traj.time, contact_array, label="contact indicator")